In [5]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

# Données reconnaissance du locuteur (Chirac/Mitterrand)

<100:5:C> Le Congo, que naguère le <nom> qualifia de "refuge pour la liberté", de "base de départ pour la libération", de "môle pour la Résistance", comment ne pas être heureux de s'y retrouver ?<br>
<100:6:C> Comment ne pas y voir un signe ?<br>
<100:7:C> Brazzaville n'est pas une capitale ordinaire.<br>
<100:8:C> Les voies de la libre disposition des peuples et de leur coopération furent explorées il y a un demi siècle, ici, à Brazzaville.<br>
<100:9:C> C'est à Brazzaville, encore, que quinze années plus tard fut proclamée la Communauté.<br>
<100:10:C> A Brazzaville, que fut scellée la première union régionale des pays africains francophones.<br>
<100:11:C> A Brazzaville, que l'Afrique de demain se dessine.<br>
<100:12:M> Je ne sais ni pourquoi ni comment on s'est opposé il y a quelques douze années - douze ou treize ans - à la création de l'Université technologique.<br>
<100:13:M> C'est vrai qu'il y a très souvent dans notre pays un refus de regarder droit devant soi, comme un souhait d'en rester là, une certaine peur du changement, c'est vrai.<br>

In [3]:
# Chargement des données:
def load_pres(fname):
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour régler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt) #recupère le label (C ou M)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt) #recupère le texte
        if lab.count('M') >0:
            alllabs.append(-1)
        else:
            alllabs.append(1) #si c'est Mitterand qui parle on met -1 si c'est Chirac, on met 1
        alltxts.append(txt)
    return alltxts,alllabs


In [6]:
re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1","<100:6:C> Comment ne pas y voir un signe ?")

'C'

In [7]:
re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1","<100:6:C> Comment ne pas y voir un signe ?")

' Comment ne pas y voir un signe ?'

In [4]:
fname = "./corpus.tache1.learn.utf8"
alltxts,alllabs = load_pres(fname)

In [8]:
print(len(alltxts),len(alllabs))
print(alltxts[0])
print(alllabs[0])
print(alltxts[13])
print(alllabs[13])
print(alltxts[-1])
print(alllabs[-1])


57413 57413
 Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.

1
 Et ce sentiment n'est pas également partagé, monsieur le maire, mais enfin il existe de tous côtés.

-1
 Je compte sur vous.

1


# Données classification de sentiments (films)

In [9]:
def load_movies(path2data): # 1 classe par répertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un répertoire
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg répertoire = cht classe

    return alltxts,labs


In [10]:
path = "./movies1000/"

alltxts,alllabs = load_movies(path)

In [11]:
print(len(alltxts),len(alllabs))
print(alltxts[0])
print(alllabs[0])
print(alltxts[-1])
print(alllabs[-1])

2000 2000
armageddon , in itself , symbolizes everything that is wrong in modern filmmaking . 
stories have been replaced with special effects ; character development gets overshadowed by bad dialogue ; plotting consists of a bunch of shit getting blown up . 
armageddon is as stupid , as loud and as shallow as any movie you'll see come out this summer , or maybe even any other summer . 
but i loved every freaking minute of it . 
believe me , i'm just as shocked as you are . 
hell , i don't even know why i went to see it in the first place . 
the previews were so annoying that i predicted this was going to be the worst film of the year , or at least in the running . 
i'm sorry , but " somebody dial 911 ! ! ! " 
isn't quite the tagging that's going to sell a movie . 
it isn't too wise either to market the film using the movie's stupidest lines ( " beam me up scotty " - yeah , that sure is great writing . . . ) . 
i mean , let's face it ; armageddon's previews rival the truman show's as b

# A) Transformation paramétrique du texte (pre-traitements)

Vous devez tester, par exemple, les cas suivants:
- transformation en minuscule ou pas
- suppression de la ponctuation
- transformation des mots entièrement en majuscule en marqueurs spécifiques
- suppression des chiffres ou pas
- conservation d'une partie du texte seulement (seulement la première ligne = titre, seulement la dernière ligne = résumé, ...)
- stemming
- ...


Vérifier systématiquement sur un exemple ou deux le bon fonctionnement des méthodes sur deux documents (au moins un de chaque classe).

In [74]:
import re
import string
from sklearn.feature_extraction.text import CountVectorizer

def preprocessPerso(text): #suppression de la ponctuation
    punc = string.punctuation
    punc += '\n\r\t'
    #suppression de la ponctuation
    text = text.translate(str.maketrans(punc, ' '*len(punc)))
    #suppression des chiffres
    text = re.sub(r"[0-9]","",text)
    return text

#Verification du filtrage
print(preprocessPerso(alltxts[0]))
max_df = 5
stop_words = ["the","a","and","is","if","i","in","on","of","ll","do","does","any","all","at","it","as","you"]
vectorizer = CountVectorizer(preprocessor=preprocessPerso,lowercase=True,\
                             strip_accents='unicode',max_features=30000,stop_words=stop_words)


X = vectorizer.fit_transform(alltxts) #matrice avec id ligne, id mot dans la ligne et frequence
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire[:50],vocabulaire[-50:]) #donne les mots du vocabulaire
print("----------------- Liste des mots du 1er document ------------------------")
print(X[0])
#print("nombre de mots dans le 1er document:",X[0].size)
#print("----------------- Liste des mots du 2eme document ------------------------")
#print(X[1])
#print("nombre de mots dans le 2eme document:",X[1].size)

#print(X.toarray())

armageddon   in itself   symbolizes everything that is wrong in modern filmmaking    stories have been replaced with special effects   character development gets overshadowed by bad dialogue   plotting consists of a bunch of shit getting blown up    armageddon is as stupid   as loud and as shallow as any movie you ll see come out this summer   or maybe even any other summer    but i loved every freaking minute of it    believe me   i m just as shocked as you are    hell   i don t even know why i went to see it in the first place    the previews were so annoying that i predicted this was going to be the worst film of the year   or at least in the running    i m sorry   but   somebody dial           isn t quite the tagging that s going to sell a movie    it isn t too wise either to market the film using the movie s stupidest lines     beam me up scotty     yeah   that sure is great writing            i mean   let s face it   armageddon s previews rival the truman show s as being some of 

In [75]:
#Mots conservés suite à notre filtrage
mots = X.toarray()[0]
for i in range(len(mots)):
    if mots[i]!=0:
        print(vocabulaire[i],end=" ")

about accent affleck also always amount an annihilation annoying another apollo are armageddon asteroid attend bad basically bay be beam been before being believe belonging ben beyond bigwig billy blast blockbuster blown bob bruce bunch buscemi but by calls can case character characters checking children close coarse coin come completely consists contrived cool course dan days definitely deliver despite development device dial dialogue did doesn doing don drillers earth ed effects eight eighteen either employing enjoyed even every everything example face feet film filmmaking films first flip flying former found freaking free front full genius get gets getting go going great group hardly harris harry have he hell horny how huge huh hundred impact implant intensity interesting into introduced isn itself jet join just justice kennedy kind know leader least leaving let lines little liv loud loved lovers made market maybe mayhem mcdonald me mean melodrama michael minute modern mostly movie 

# B) Extraction du vocabulaire (BoW)

- **Exploration préliminaire des jeux de données**
    - Quelle est la taille d'origine du vocabulaire?
    - Que reste-t-il si on ne garde que les 100 mots les plus fréquents? [word cloud]
    - Quels sont les 100 mots dont la fréquence documentaire est la plus grande? [word cloud]
    - Quels sont les 100 mots les plus discriminants au sens de odds ratio? [word cloud]
    - Quelle est la distribution d'apparition des mots (Zipf)
    - Quels sont les 100 bigrammes/trigrammes les plus fréquents?

- **Variantes de BoW**
    - TF-IDF
    - Réduire la taille du vocabulaire (min_df, max_df, max_features)
    - BoW binaire
    - Bi-grams, tri-grams
    - **Quelles performances attendre ? Quels sont les avantages et les inconvénients des ces variantes?**

# C) Modèles de Machine Learning

Quelle est la taille d'origine du vocabulaire?

In [6]:
fname = "./corpus.tache1.learn.utf8"
alltxts,alllabs = load_pres(fname)

In [18]:
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode

def preprocessPerso(text): 
    #suppression de la ponctuation
    punc = string.punctuation
    punc += '\n\r\t'
    text = text.translate(str.maketrans(punc, ' '*len(punc)))
    #suppression des chiffres
    text = re.sub(r"[0-9]","",text)
    #suppression des accents
    text = unidecode(text)
    #mise en minuscule
    return text.lower()

In [19]:
vectorizer = CountVectorizer(preprocessor=preprocessPerso)

X = vectorizer.fit_transform(alltxts) #matrice avec id ligne, id mot dans la ligne et frequence
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire[:50],vocabulaire[27000:-1]) #donne les mots du vocabulaire

Taille du vocabulaire= 27055
['aaron' 'ab' 'abaissaient' 'abaissant' 'abaisse' 'abaissement'
 'abaissent' 'abaisser' 'abandon' 'abandonne' 'abandonnee' 'abandonnees'
 'abandonnent' 'abandonner' 'abandonnerai' 'abandonnes' 'abandonnons'
 'abat' 'abattage' 'abattait' 'abatte' 'abattent' 'abattre' 'abattu'
 'abattues' 'abbaye' 'abbayes' 'abbes' 'abc' 'abces' 'abdication'
 'abdiquer' 'abdou' 'abdul' 'abeba' 'aberrant' 'aberration' 'aberrations'
 'abi' 'abidjan' 'abime' 'abimee' 'abimer' 'abimes' 'abitur' 'abject'
 'abjection' 'abm' 'abnegation' 'abois'] ['xviie' 'xviieme' 'xviiie' 'xviiieme' 'xxe' 'xxeme' 'xxes' 'xxie'
 'xxieme' 'xxiie' 'xxiieme' 'yacireta' 'yacyreta' 'yalta' 'yamoussoukro'
 'yangzhou' 'yantala' 'yaounde' 'yaourt' 'yaourts' 'yemen' 'yen' 'yenenga'
 'yersin' 'yeux' 'ylang' 'yonne' 'york' 'yorktown' 'yoruba' 'you'
 'yougoslave' 'yougoslaves' 'yougoslavie' 'young' 'yvelines' 'yvon'
 'zagreb' 'zambie' 'zelande' 'zenith' 'zeniths' 'zerbo' 'zero' 'zeste'
 'zeus' 'zinder' 'zitoun

Que reste-t-il si on ne garde que les 100 mots les plus fréquents? [word cloud]


In [20]:
vectorizer = CountVectorizer(preprocessor=preprocessPerso,max_features=100)

X = vectorizer.fit_transform(alltxts)
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire) #donne les mots du vocabulaire

Taille du vocabulaire= 100
['ai' 'au' 'aujourd' 'aussi' 'autres' 'aux' 'avec' 'avenir' 'avez' 'avons'
 'bien' 'ce' 'cela' 'ces' 'cette' 'ceux' 'chacun' 'comme' 'dans' 'date'
 'de' 'depuis' 'des' 'deux' 'developpement' 'dire' 'doit' 'dont' 'du'
 'elle' 'en' 'encore' 'ensemble' 'entre' 'est' 'et' 'etat' 'ete' 'etre'
 'europe' 'faire' 'fait' 'faut' 'francais' 'france' 'hui' 'ici' 'il' 'ils'
 'je' 'la' 'le' 'les' 'leur' 'leurs' 'mais' 'meme' 'monde' 'monsieur' 'ne'
 'nom' 'nos' 'notre' 'nous' 'on' 'ont' 'ou' 'paix' 'par' 'pas' 'pays'
 'peut' 'plus' 'politique' 'pour' 'president' 'qu' 'que' 'qui' 'sa' 'sans'
 'se' 'ses' 'si' 'son' 'sont' 'sur' 'temps' 'tous' 'tout' 'toute' 'toutes'
 'tres' 'un' 'une' 'union' 'vie' 'vos' 'votre' 'vous']


Ne conserve que les 100 mots les plus fréquents qui ne sont pas discriminants.
On peut donc les éliminer à l'aide de l'option stopwords.

In [24]:
stopwords = ['ai' 'au' 'aujourd' 'aussi' 'autres' 'aux' 'avec' 'avenir' 'avez' 'avons'
 'bien' 'ce' 'cela' 'ces' 'cette' 'ceux' 'chacun' 'comme' 'dans' 'date'
 'de' 'depuis' 'des' 'deux' 'developpement' 'dire' 'doit' 'dont' 'du'
 'elle' 'en' 'encore' 'ensemble' 'entre' 'est' 'et' 'etat' 'ete' 'etre'
 'europe' 'faire' 'fait' 'faut' 'francais' 'france' 'hui' 'ici' 'il' 'ils'
 'je' 'la' 'le' 'les' 'leur' 'leurs' 'mais' 'meme' 'monde' 'monsieur' 'ne'
 'nom' 'nos' 'notre' 'nous' 'on' 'ont' 'ou' 'paix' 'par' 'pas' 'pays'
 'peut' 'plus' 'politique' 'pour' 'president' 'qu' 'que' 'qui' 'sa' 'sans'
 'se' 'ses' 'si' 'son' 'sont' 'sur' 'temps' 'tous' 'tout' 'toute' 'toutes'
 'tres' 'un' 'une' 'union' 'vie' 'vos' 'votre' 'vous']

In [26]:
vectorizer = CountVectorizer(preprocessor=preprocessPerso,stop_words=stopwords)

X = vectorizer.fit_transform(alltxts) #matrice avec id ligne, id mot dans la ligne et frequence
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire[:50],vocabulaire[27000:-1]) #donne les mots du vocabulaire

Taille du vocabulaire= 27055
['aaron' 'ab' 'abaissaient' 'abaissant' 'abaisse' 'abaissement'
 'abaissent' 'abaisser' 'abandon' 'abandonne' 'abandonnee' 'abandonnees'
 'abandonnent' 'abandonner' 'abandonnerai' 'abandonnes' 'abandonnons'
 'abat' 'abattage' 'abattait' 'abatte' 'abattent' 'abattre' 'abattu'
 'abattues' 'abbaye' 'abbayes' 'abbes' 'abc' 'abces' 'abdication'
 'abdiquer' 'abdou' 'abdul' 'abeba' 'aberrant' 'aberration' 'aberrations'
 'abi' 'abidjan' 'abime' 'abimee' 'abimer' 'abimes' 'abitur' 'abject'
 'abjection' 'abm' 'abnegation' 'abois'] ['xviie' 'xviieme' 'xviiie' 'xviiieme' 'xxe' 'xxeme' 'xxes' 'xxie'
 'xxieme' 'xxiie' 'xxiieme' 'yacireta' 'yacyreta' 'yalta' 'yamoussoukro'
 'yangzhou' 'yantala' 'yaounde' 'yaourt' 'yaourts' 'yemen' 'yen' 'yenenga'
 'yersin' 'yeux' 'ylang' 'yonne' 'york' 'yorktown' 'yoruba' 'you'
 'yougoslave' 'yougoslaves' 'yougoslavie' 'young' 'yvelines' 'yvon'
 'zagreb' 'zambie' 'zelande' 'zenith' 'zeniths' 'zerbo' 'zero' 'zeste'
 'zeus' 'zinder' 'zitoun

In [32]:
# French stop words: nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
final_stopwords_list = stopwords.words('french')
#print(final_stopwords_list)
vectorizer = CountVectorizer(preprocessor=preprocessPerso,stop_words=final_stopwords_list)
#vectorizer = TfidfVectorizer(stop_words=final_stopwords_list)
X = vectorizer.fit_transform(alltxts)
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire[:50],vocabulaire[26000:-1]) #donne les mots du vocabulaire

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nilsbarrellon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Taille du vocabulaire= 26943
['aaron' 'ab' 'abaissaient' 'abaissant' 'abaisse' 'abaissement'
 'abaissent' 'abaisser' 'abandon' 'abandonne' 'abandonnee' 'abandonnees'
 'abandonnent' 'abandonner' 'abandonnerai' 'abandonnes' 'abandonnons'
 'abat' 'abattage' 'abattait' 'abatte' 'abattent' 'abattre' 'abattu'
 'abattues' 'abbaye' 'abbayes' 'abbes' 'abc' 'abces' 'abdication'
 'abdiquer' 'abdou' 'abdul' 'abeba' 'aberrant' 'aberration' 'aberrations'
 'abi' 'abidjan' 'abime' 'abimee' 'abimer' 'abimes' 'abitur' 'abject'
 'abjection' 'abm' 'abnegation' 'abois'] ['twinauter' 'type' 'types' 'typhon' 'typhons' 'typique' 'typiquement'
 'typographie' 'tyrannie' 'tyrannies' 'tyrannique' 'tyrans' 'tyrrhenienne'
 'tzigane' 'ubiquite' 'udf' 'ue' 'ueo' 'ugag' 'uhlans' 'uicn' 'ukraine'
 'ukrainien' 'ukrainienne' 'ukrainiens' 'ulterieurement' 'ultimatum'
 'ultime' 'ultimes' 'ultra' 'ultramarine' 'ultramarines' 'ultramodernes'
 'ultraperipherie' 'ultraperipherique' 'ultraperipheriques' 'ultras'
 'ulysse' 'unaf

Quels sont les 100 mots dont la fréquence documentaire est la plus grande?

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

use_idf=True
smooth_idf=True
sublinear_tf=False

vectorizer = TfidfVectorizer(preprocessor=preprocessPerso,use_idf= use_idf, smooth_idf=smooth_idf, sublinear_tf=sublinear_tf,max_features=100)
X = vectorizer.fit_transform(alltxts)
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire) #donne les mots du vocabulaire

Taille du vocabulaire= 100
['ai' 'au' 'aujourd' 'aussi' 'autres' 'aux' 'avec' 'avenir' 'avez' 'avons'
 'bien' 'ce' 'cela' 'ces' 'cette' 'ceux' 'chacun' 'comme' 'dans' 'date'
 'de' 'depuis' 'des' 'deux' 'developpement' 'dire' 'doit' 'dont' 'du'
 'elle' 'en' 'encore' 'ensemble' 'entre' 'est' 'et' 'etat' 'ete' 'etre'
 'europe' 'faire' 'fait' 'faut' 'francais' 'france' 'hui' 'ici' 'il' 'ils'
 'je' 'la' 'le' 'les' 'leur' 'leurs' 'mais' 'meme' 'monde' 'monsieur' 'ne'
 'nom' 'nos' 'notre' 'nous' 'on' 'ont' 'ou' 'paix' 'par' 'pas' 'pays'
 'peut' 'plus' 'politique' 'pour' 'president' 'qu' 'que' 'qui' 'sa' 'sans'
 'se' 'ses' 'si' 'son' 'sont' 'sur' 'temps' 'tous' 'tout' 'toute' 'toutes'
 'tres' 'un' 'une' 'union' 'vie' 'vos' 'votre' 'vous']


Les 100 mots les plus fréquents se sont pas intéressant au niveau sémantique. On peut les supprimer en les ajoutant à la stop_word liste

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

use_idf=True
smooth_idf=True
sublinear_tf=False
from nltk.corpus import stopwords
final_stopwords_list = stopwords.words('french')
stop_words = stopwords.words('french')+['ai' 'au' 'aujourd' 'aussi' 'autres' 'aux' 'avec' 'avenir' 'avez' 'avons'
 'bien' 'ce' 'cela' 'ces' 'cette' 'ceux' 'chacun' 'comme' 'dans' 'date'
 'de' 'depuis' 'des' 'deux' 'developpement' 'dire' 'doit' 'dont' 'du'
 'elle' 'en' 'encore' 'ensemble' 'entre' 'est' 'et' 'etat' 'ete' 'etre'
 'europe' 'faire' 'fait' 'faut' 'francais' 'france' 'hui' 'ici' 'il' 'ils'
 'je' 'la' 'le' 'les' 'leur' 'leurs' 'mais' 'meme' 'monde' 'monsieur' 'ne'
 'nom' 'nos' 'notre' 'nous' 'on' 'ont' 'ou' 'paix' 'par' 'pas' 'pays'
 'peut' 'plus' 'politique' 'pour' 'president' 'qu' 'que' 'qui' 'sa' 'sans'
 'se' 'ses' 'si' 'son' 'sont' 'sur' 'temps' 'tous' 'tout' 'toute' 'toutes'
 'tres' 'un' 'une' 'union' 'vie' 'vos' 'votre' 'vous']
vectorizer = TfidfVectorizer(preprocessor=preprocessPerso,use_idf= use_idf, smooth_idf=smooth_idf, \
                             sublinear_tf=sublinear_tf,stop_words=stop_words)
X = vectorizer.fit_transform(alltxts)
vocabulaire = vectorizer.get_feature_names_out()
print("Taille du vocabulaire=",len(vocabulaire))
print(vocabulaire[:100],vocabulaire[-100:]) #donne les mots du vocabulaire

Taille du vocabulaire= 26943
['aaron' 'ab' 'abaissaient' 'abaissant' 'abaisse' 'abaissement'
 'abaissent' 'abaisser' 'abandon' 'abandonne' 'abandonnee' 'abandonnees'
 'abandonnent' 'abandonner' 'abandonnerai' 'abandonnes' 'abandonnons'
 'abat' 'abattage' 'abattait' 'abatte' 'abattent' 'abattre' 'abattu'
 'abattues' 'abbaye' 'abbayes' 'abbes' 'abc' 'abces' 'abdication'
 'abdiquer' 'abdou' 'abdul' 'abeba' 'aberrant' 'aberration' 'aberrations'
 'abi' 'abidjan' 'abime' 'abimee' 'abimer' 'abimes' 'abitur' 'abject'
 'abjection' 'abm' 'abnegation' 'abois' 'abolie' 'abolir' 'abolirait'
 'abolissent' 'abolit' 'abolition' 'abominable' 'abominables'
 'abondamment' 'abondance' 'abondante' 'abondantes' 'abonde' 'abondent'
 'abonder' 'abord' 'abordait' 'abordant' 'aborde' 'abordee' 'abordees'
 'abordent' 'aborder' 'abordera' 'aborderai' 'aborderait' 'aborderons'
 'aborderont' 'abordes' 'abordez' 'abordions' 'abordons' 'abords' 'abou'
 'abouti' 'aboutir' 'aboutissait' 'aboutissant' 'aboutisse'
 'abou

Quels sont les 100 mots les plus discriminants au sens de odds ratio?

In [46]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Exemple de corpus (à remplacer par tes données)
corpus1 = [alltxts[i] for i in range(len(alltxts)) if alllabs[i]==1]

corpus2 = [alltxts[i] for i in range(len(alltxts)) if alllabs[i]==-1]

# Nettoyage et vectorisation
vectorizer = CountVectorizer(token_pattern=r'\b[a-zA-Zàâäéèêëîïôöùûüçœ]{2,}\b')
X = vectorizer.fit_transform(corpus1 + corpus2)
features = vectorizer.get_feature_names_out()

# Fréquences par corpus
X1 = X[:len(corpus1)].sum(axis=0).A1
X2 = X[len(corpus1):].sum(axis=0).A1

# Calcul de l'odds ratio
freq1 = pd.Series(X1, index=features)
freq2 = pd.Series(X2, index=features)

# Éviter les divisions par zéro
freq1 = freq1 + 1
freq2 = freq2 + 1

odds_ratio = (freq1 / (freq1.sum() - freq1)) / (freq2 / (freq2.sum() - freq2))

# Trier par odds ratio (descendant)
top_100 = odds_ratio.sort_values(ascending=False).head(100)

print(top_100)

mondialisation    76.560328
euro              65.929850
partenariat       38.605431
attentes          37.948305
constituent       22.333734
                    ...    
compatriotes       8.217135
chaleureux         8.167359
moteur             8.022537
ueo                8.022286
ardeur             8.022286
Length: 100, dtype: float64


# C) Modèles de Machine Learning¶

## 1) Métriques d'évaluation

Il faudra utiliser des métriques d'évaluation pertinentes suivant la tâche et l'équilibrage des données :
- Accuracy
- Courbe ROC, AUC, F1-score

## 2) Variantes sur les stratégies d'entraînement

- **Sur-apprentissage**. Les techniques sur lesquelles nous travaillons étant sujettes au sur-apprentissage: trouver le paramètre de régularisation dans la documentation et optimiser ce paramètre au sens de la métrique qui vous semble la plus appropriée (cf question précédente).

 <br>
- **Equilibrage des données**. Un problème reconnu comme dur dans la communauté est celui de l'équilibrage des classes (*balance* en anglais). Que faire si les données sont à 80, 90 ou 99% dans une des classes?
Le problème est dur mais fréquent; les solutions sont multiples mais on peut isoler 3 grandes familles de solution.

1. Ré-équilibrer le jeu de données: supprimer des données dans la classe majoritaire et/ou sur-échantilloner la classe minoritaire.<BR>
   $\Rightarrow$ A vous de jouer pour cette technique
1. Changer la formulation de la fonction de coût pour pénaliser plus les erreurs dans la classe minoritaire:
soit une fonction $\Delta$ mesurant les écarts entre $f(x_i)$ et $y_i$
$$C = \sum_i  \alpha_i \Delta(f(x_i),y_i), \qquad \alpha_i = \left\{
\begin{array}{ll}
1 & \text{si } y_i \in \text{classe majoritaire}\\
B>1 & \text{si } y_i \in \text{classe minoritaire}\\
\end{array} \right.$$
<BR>
   $\Rightarrow$ Les SVM et d'autres approches sklearn possèdent des arguments pour régler $B$ ou $1/B$... Ces arguments sont utiles mais pas toujours suffisant.
1. Courbe ROC et modification du biais. Une fois la fonction $\hat y = f(x)$ apprise, il est possible de la *bidouiller* a posteriori: si toutes les prédictions $\hat y$ sont dans une classe, on va introduire $b$ dans $\hat y = f(x) + b$ et le faire varier jusqu'à ce qu'un des points change de classe. On peut ensuite aller de plus en plus loin.
Le calcul de l'ensemble des scores associés à cette approche mène directement à la courbe ROC.

**Note:** certains classifieurs sont intrinsèquement plus résistante au problème d'équilibrage, c'est par exemple le cas des techniques de gradient boosting que vous verrez l'an prochain.

## 3) Post-processing sur les données Président

Pour la tâche de reconnaissance de locuteur, des phrases successives sont souvent associés à un même locuteur. Voir par exemples les 100 premiers labels de la base d'apprentissage.

In [ ]:
fname = "./datasets/AFDpresidentutf8/corpus.tache1.learn.utf8"
alltxts,alllabs = load_pres(fname)

plt.figure()
plt.plot(list(range(len(alllabs[0:100]))),alllabs[0:100])

**Une méthode de post-traitement pour améliorer les résultats consistent à lisser les résultats de la prédictions d'une phrases par les prédictions voisines, en utilisant par exemple une convolution par une filtre Gaussien. Compléter la fonction ci-dessous et tester l'impact de ce lissage sur les performances.**

In [ ]:
def gaussian_smoothing(pred, size):
     # LISSAGE par un filtre Gaussien de taille size - vous pouvez utiliser np.convolve
    return 0


## 4) Estimer les performances de généralisation d'une méthodes
**Ce sera l'enjeu principal du projet : vous disposez d'un ensemble de données, et vous évaluerez les performances sur un ensemble de test auquel vous n'avez pas accès. Il faut donc être capable d'estimer les performances de généralisation du modèles à partir des données d'entraînement.**


Avant de lancer de grandes expériences, il faut se construire une base de travail solide en étudiant les questions suivantes:

- Combien de temps ça prend d'apprendre un classifieur NB/SVM/RegLog sur ces données en fonction de la taille du vocabulaire?
- La validation croisée est-elle nécessaire? Est ce qu'on obtient les mêmes résultats avec un simple *split*?
- La validation croisée est-elle stable? A partir de combien de fold (travailler avec différentes graines aléatoires et faire des statistiques basiques)?